# Rooflines 
> Plotting rooflines with Altair

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

In [ ]:
#hide
import numpy as np
import pandas as pd

import altair as alt

In [ ]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
tableOverviewExperiments(['data/overview_experiments_imagenet.csv'])

# Rooflines for all hardware platforms with Neural Networks arithmetic Intensity

In [ ]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
rooflines(pd.read_csv("data/processed_csv/rooflines_hardware_neural_networks.csv"), 'All')

# Performance Predictions for IMAGENET, MNIST and CIFAR

In [ ]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
heatmap(pd.read_csv("data/processed_csv/performance_prediction_imagenet_mnist_cifar10.csv"), 
        'red', 
        'Performance Predictions for IMAGENET, MNIST and CIFAR10')